In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import requests
import io 
import zipfile
import xml.etree.ElementTree as et 
import json

In [12]:
api_num = pd.read_csv('./data/dart_api_key.txt', sep='\s', header=None)

'3f19505ff7834fc28b5471e5a0b01839b29e5d16'

In [13]:
def get_corpcode(crtfc_key):
    params = {'crtfc_key':crtfc_key}
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    items_en = ["corp_code", "corp_name", "stock_code", "modify_date"]
    
    ## Column명을 한글로 하려면 아래부분 쌍따옴표안을 수정하면 됨 
    items_kr = ["corp_code", "corp_name", "stock_code", "modify_date"] 
    results = requests.get(url,params=params) 
    uzfile = zipfile.ZipFile(io.BytesIO(results.content))
    final = uzfile.open(uzfile.namelist()[0])
    root = et.fromstring(final.read().decode('utf-8'))
    data = [] 
    for child in root: 
        if len(child.find('stock_code').text.strip()) > 1: 
            data.append([]) 
            for item in items_en: 
                data[-1].append(child.find(item).text) 
    df = pd.DataFrame(data, columns = items_kr) 
    ##print(df) 
    return df

In [14]:
crtfc_key = api_num[0][0]
code_df = get_corpcode(crtfc_key)
code_df.head()

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630


In [15]:
code_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3418 entries, 0 to 3417
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp_code    3418 non-null   object
 1   corp_name    3418 non-null   object
 2   stock_code   3418 non-null   object
 3   modify_date  3418 non-null   object
dtypes: object(4)
memory usage: 106.9+ KB


In [19]:
company_name_df = pd.read_csv('./data/시총500(코스피).txt', sep='\s+', header=None,
                                names=['column_name'])

company_name_df

,column_name
0,강원랜드
1,고려아연
2,금호석유
3,기아
4,기업은행
...,...
495,모토닉
496,에이프로젠
497,신세계
498,태림포장


In [20]:
company_names = company_name_df['column_name'].values.tolist()
print(company_names[:20])

['강원랜드', '고려아연', '금호석유', '기아', '기업은행', '넷마블', '대한항공', '두산밥캣', '두산중공업', '롯데케미칼', '맥쿼리인프라', '메리츠금융지주', '메리츠증권', '메리츠화재', '미래에셋증권', '삼성물산', '삼성바이오로직스', '삼성생명', '삼성에스디에스', '삼성엔지니어링']


In [30]:
df_cos = code_df[code_df['corp_name'].isin(company_names)]
df_cos.head(10)

,corp_code,corp_name,stock_code,modify_date
313,00375302,우리금융지주,053000,20170630
402,00139418,알루코,021570,20170630
486,00311030,미래에셋증권,037620,20170630
524,00126229,삼성물산,000830,20170630
580,00144155,SK,003600,20170630
611,00686068,신성이엔지,104110,20171208
667,00600013,맵스리얼티1,094800,20190405
733,00539274,대상홀딩스,084690,20210216
736,00435297,맥쿼리인프라,088980,20200715
741,01325979,세아제강,306200,20210218


In [31]:
df_cos = df_cos.sort_values('corp_name', ascending=True)
len(df_cos)

399

In [32]:
df_cos.head()

,corp_code,corp_name,stock_code,modify_date
1843,00219097,BGF,027410,20211207
2068,01263022,BGF리테일,282330,20220106
1451,00858364,BNK금융지주,138930,20220110
1986,00148540,CJ,001040,20220215
2308,00113410,CJ대한통운,000120,20211203


In [47]:
code_df.to_excel('고유번호3.xlsx')

In [69]:
api_key = '-'
corp_code = '00219097'
search_year = '2017'
first_quarter = '11013'
half = '11012'
third_quarter = '11014'
one_year = '11011'

url = f'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={search_year}&reprt_code={first_quarter}'
response = requests.get(url)

response

<Response [200]>

In [67]:
from tqdm import tqdm_notebook

api_key = '-'
search_year = ['2016', '2017', '2017', '2019', '2020']
quarter_list = ['11013', '11012', '11014', '11011']

data = []
for corp_code in tqdm_notebook(df_cos['corp_code']):
    for year in search_year:
        for quarter in quarter_list:
            url = f'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={search_year}&reprt_code={first_quarter}'
            response = requests.get(url)
            company_code = response['list'][0]['corp_code']
            liquid_asset = response['list'][0]['thstrm_amount']
            total_asset = response['list'][2]['thstrm_amount']
            liquid_debt = response['list'][3]['thstrm_amount']
            total_debt = response['list'][5]['thstrm_amount']
            
            
            finance_data1 = {
                '유동자산' : liquid_asset,
                '총자산' : total_asset,
                '유동부채' : liquid_debt,
                '총부채' : total_debt
            }

            data.append(finance_data1)
            
            if response['list'][6]['account_nm'] == '자본금':
                ordinary_share = response['list'][6]['thstrm_amount']
                retained_earning = response['list'][7]['thstrm_amount']
                total_equity = response['list'][8]['thstrm_amount']
                revenue = response['list'][9]['thstrm_amount']
                operating_profit = response['list'][10]['thstrm_amount']
                net_income = response['list'][12]['thstrm_amount']

                finance_data2 = {
                    '자본금' : ordinary_share,
                    '이익잉여금' : retained_earning,
                    '총자본' : total_equity,
                    '매출액' : revenue,
                    '영업이익' : operating_profit,
                    '당기순이익' : net_income
                }

                data.append(finance_data2)

            else:
                retained_earning = response['list'][6]['thstrm_amount']
                total_equity = response['list'][7]['thstrm_amount']
                revenue = response['list'][8]['thstrm_amount']
                operating_profit = response['list'][9]['thstrm_amount']
                net_income =  response['list'][11]['thstrm_amount']

                finance_data3 = {
                    '이익잉여금' : retained_earning,
                    '총자본' : total_equity,
                    '매출액' : revenue,
                    '영업이익' : operating_profit,
                    '당기순이익' : net_income
                }
                data.append(finance_data3)

df_finance = pd.DataFrame(data)
df_finance.head()

  0%|          | 0/399 [00:00<?, ?it/s]

,유동자산,총자산,유동부채,총부채,이익잉여금,총자본,매출액,영업이익,당기순이익
0,"986,585,650,559","1,955,808,596,927","771,067,992,964","1,022,440,656,635",NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,"794,887,828,227","933,367,940,292","1,215,645,931,124","39,634,034,678","32,090,130,727"
2,"986,585,650,559","1,955,808,596,927","771,067,992,964","1,022,440,656,635",NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,"794,887,828,227","933,367,940,292","1,215,645,931,124","39,634,034,678","32,090,130,727"
4,"986,585,650,559","1,955,808,596,927","771,067,992,964","1,022,440,656,635",NaN,NaN,NaN,NaN,NaN


In [68]:
df_finance.to_excel('코스피기업 재무제표.xlsx')

In [52]:
response_first = response.json()

In [62]:
# 1분기, 반기, 3분기의 경우

print(response_first['list'][0]['account_nm'], response_first['list'][0]['thstrm_amount'])
print(response_first['list'][2]['account_nm'], response_first['list'][2]['thstrm_amount'])
print(response_first['list'][3]['account_nm'], response_first['list'][3]['thstrm_amount'])
print(response_first['list'][5]['account_nm'], response_first['list'][5]['thstrm_amount'])
print(response_first['list'][6]['account_nm'], response_first['list'][6]['thstrm_amount'])
print(response_first['list'][7]['account_nm'], response_first['list'][7]['thstrm_amount'])
print(response_first['list'][8]['account_nm'], response_first['list'][8]['thstrm_amount'])
print(response_first['list'][9]['account_nm'], response_first['list'][9]['thstrm_amount'])
print(response_first['list'][11]['account_nm'], response_first['list'][11]['thstrm_amount'])


유동자산 986,585,650,559
자산총계 1,955,808,596,927
유동부채 771,067,992,964
부채총계 1,022,440,656,635
이익잉여금 794,887,828,227
자본총계 933,367,940,292
매출액 1,215,645,931,124
영업이익 39,634,034,678
당기순이익 32,090,130,727


In [57]:
url = f'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={search_year}&reprt_code={half}'
response = requests.get(url)

response_half = response.json()

In [63]:
# 1분기, 반기, 3분기의 경우

print(response_half['list'][0]['account_nm'], response_half['list'][0]['thstrm_amount'])
print(response_half['list'][2]['account_nm'], response_half['list'][2]['thstrm_amount'])
print(response_half['list'][3]['account_nm'], response_half['list'][3]['thstrm_amount'])
print(response_half['list'][5]['account_nm'], response_half['list'][5]['thstrm_amount'])
print(response_half['list'][6]['account_nm'], response_half['list'][6]['thstrm_amount'])
print(response_half['list'][7]['account_nm'], response_half['list'][7]['thstrm_amount'])
print(response_half['list'][8]['account_nm'], response_half['list'][8]['thstrm_amount'])
print(response_half['list'][9]['account_nm'], response_half['list'][9]['thstrm_amount'])
print(response_half['list'][11]['account_nm'], response_half['list'][11]['thstrm_amount'])


유동자산 1,106,561,978,766
자산총계 2,088,182,729,939
유동부채 850,125,912,032
부채총계 1,094,076,126,281
이익잉여금 855,742,914,552
자본총계 994,106,603,658
매출액 1,412,995,579,893
영업이익 74,110,709,511
당기순이익 61,212,937,931


In [60]:
url = f'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={search_year}&reprt_code={third_quarter}'
response = requests.get(url)

response_third = response.json()

In [64]:
# 1분기, 반기, 3분기의 경우

print(response_third['list'][0]['account_nm'], response_third['list'][0]['thstrm_amount'])
print(response_third['list'][2]['account_nm'], response_third['list'][2]['thstrm_amount'])
print(response_third['list'][3]['account_nm'], response_third['list'][3]['thstrm_amount'])
print(response_third['list'][5]['account_nm'], response_third['list'][5]['thstrm_amount'])
print(response_third['list'][6]['account_nm'], response_third['list'][6]['thstrm_amount'])
print(response_third['list'][7]['account_nm'], response_third['list'][7]['thstrm_amount'])
print(response_third['list'][8]['account_nm'], response_third['list'][8]['thstrm_amount'])
print(response_third['list'][9]['account_nm'], response_third['list'][9]['thstrm_amount'])
print(response_third['list'][10]['account_nm'], response_third['list'][10]['thstrm_amount'])
print(response_third['list'][12]['account_nm'], response_third['list'][12]['thstrm_amount'])


유동자산 504,788,268,009
자산총계 2,218,451,758,906
유동부채 3,768,197,632,993
부채총계 4,806,090,578,739
자본금 49,547,625,000
이익잉여금 949,009,669,463
자본총계 -2,587,638,819,833
매출액 27,573,693,451
영업이익 2,913,443,474
당기순이익 93,599,478,210


In [ ]:
url = f'https://opendart.fss.or.kr/api/fnlttMultiAcnt.json?crtfc_key={api_key}&corp_code={corp_code}&bsns_year={search_year}&reprt_code={one_year}'
response = requests.get(url)

response_year = response.json()

In [56]:
# 사업보고서의 경우

print(response_year['list'][0]['account_nm'], response_year['list'][0]['thstrm_amount'])
print(response_year['list'][2]['account_nm'], response_year['list'][2]['thstrm_amount'])
print(response_year['list'][3]['account_nm'], response_year['list'][3]['thstrm_amount'])
print(response_year['list'][5]['account_nm'], response_year['list'][5]['thstrm_amount'])
print(response_year['list'][6]['account_nm'], response_year['list'][6]['thstrm_amount'])
print(response_year['list'][7]['account_nm'], response_year['list'][7]['thstrm_amount'])
print(response_year['list'][8]['account_nm'], response_year['list'][8]['thstrm_amount'])
print(response_year['list'][9]['account_nm'], response_year['list'][9]['thstrm_amount'])
print(response_year['list'][10]['account_nm'], response_year['list'][10]['thstrm_amount'])
print(response_year['list'][12]['account_nm'], response_year['list'][12]['thstrm_amount'])

유동자산 438,201,259,390
자산총계 858,118,604,869
유동부채 89,036,876,199
부채총계 132,887,657,573
자본금 32,263,719,000
이익잉여금 4,262,198,753,750
자본총계 725,230,947,296
매출액 123,842,619,339
영업이익 10,447,735,458
당기순이익 3,497,875,341,046
